# AFSK Demodulator
## Step 6: HDLC Decoder

-----

This notebook will outline the steps necessary to move the HDLC Decoder to FPGA.  After this part is complete we will integrate all of the components into a complete demodulator.

This code is part of the [AFSK Demodulator on Pynq](afsk-demodulator-fpga.ipynb) project.

As with our earlier components, we have the option of integrating the code into existing demodulator components on the FPGA or designing this as a stand-alone component.  I initially started this as an integrated design, but the complexity of the code and the difficulty diagnosing issue caused me to abandon that and do this work as a stand-alone component for now.  This was a benefit as it led to a radical change in the design and improved performance.  
During this process the CRC calculation development and testing was done in a separate module.  That work is omitted from the final published project for brevity.  The result is embedded in the HDLC module.

As with the Digital PLL, we started by borrowing code from the Mobilinkd TNC3 firmware.  We did need to write a CRC calculator.  In the TNC3, hardware IP is used to calculate the CRC.  And in the Python implementation, we use a lookup table, which would work but would be inefficient on an FPGA.

During the development, it was determined that the HDLC implementation from the TNC3 was rather inefficient on the FPGA, on the order of 1000 LUTs and having a rather high II.  

I found this publication and thought I would try to benchmark the HDLC code against it.

https://pdfs.semanticscholar.org/fd3f/6bbab3868baf3285b9bafcf0de27cbf8728d.pdf

A more traditional FPGA implementation that streams data in and out, and which does not store the packet as it is assembled would work best.  In the end, we use less than 200 luts for the HDLC, NRZI and CRC components.


## Prerequisites

At this point you are expected to have:

 * A configured PYNQ environment.
 * Vivado installed on your computer and configured for your board.
 * Experience working through the tutorials at https://pynq.readthedocs.io/.
 * Familiarized yourself with the AFSK demodulator implementation in Python.
 * Completed the previous steps of the tutorial to familiarize yourself with the process of creating a streaming interface.

## Outline

We are going to create a new FPGA IP module for this tutorial.  We will use an AXI-lite interface to pass data to the module and an AXI stream interface to retrieve it.

We will perform the following steps in this section:

 1. Create a top-level module that accepts input, sample, lock and cancel input signals, and outputs a data stream.
 1. Create a CRC module to calculate the checksum.
 1. Create an NRZI module to decode the NRZI-encoded data.
 1. Create the HDLC module to decode the HDLC data.
 1. Create a C++ test case for the above file.
 1. Generate an IP package from the code that can be used in Vivado.
 1. Create a Zynq project in Vivado that uses the IP.
 1. Export the bitstream for our project from Vivado.
 1. Use Python running on the PS to load the bitstream to the PL, and verify that it works.
 1. Integrate the FPGA module with the existing demodulator code, replacing the existing Python HDLC, CRC and NRZI code.

First we are going to generate the input and output data for the test bench.

## Test Bench Data

We are going to test just the HDLC module.  We are going to capture the output of the Digital PLL and the output of the Python HDLC module and compare the results.

#### Note

> During the development of this section, a defect was introduced into the HDLC decoder which passed our initial test case.  But when we attempted to decode a larger subset of the data, only about 1/7 of the packets were being decoded.  The test cases were extended to cover the problem.

> After the development of this code was completed, we discovered that the new code captured more packets than the previos code (about 6% more).  This indicates that there is a defect in the Python implementation.

We are going to capture 6 seconds of data starting at the 6-second mark.  This contains decodable packets.

In [4]:
import sys
sys.path.append('../base')
import numpy as np
from scipy.signal import lfiltic, lfilter, firwin
from scipy.io.wavfile import read
from DigitalPLL import DigitalPLL
from HDLC import HDLC

audio_file = read('../base/TNC_Test_Ver-1.101-26400-1min.wav')
sample_rate = audio_file[0]
audio_data = audio_file[1][26400*6:26400*12]

bpf_coeffs = np.array(firwin(141, [1100.0/(sample_rate/2), 2300.0/(sample_rate/2)], width = None,
        pass_zero = False, scale = True, window='hann') * 32768, dtype=int)

lpf_coeffs = np.array(firwin(101, [760.0/(sample_rate/2)], width = None,
        pass_zero = True, scale = True, window='hann') * 32768, dtype=int)

pll = DigitalPLL(sample_rate, 1200.0)

class fir_filter(object):
    def __init__(self, coeffs):
        self.coeffs = coeffs
        self.zl = lfiltic(self.coeffs, 32768, [], [])
    def __call__(self, data):
        result, self.zl = lfilter(self.coeffs, 32768, data, -1, self.zl)
        return result

bpf = fir_filter(bpf_coeffs)
lpf = fir_filter(lpf_coeffs)

delay = 12

f = bpf(audio_data)
c = np.array([int(x >= 0) for x in f])
# Delay the data
d = np.append(np.zeros(delay, dtype=int), np.array(c[:0-delay], dtype=int))
# XOR the digitized data with the delayed version
x = np.logical_xor(c, d)
l = lpf(x * 2 - 1)
comp = np.array([int(x >= 0) for x in l])

locked = np.zeros(len(comp), dtype=int)
sample = np.zeros(len(comp), dtype=int)

for i in range(len(comp)):
    sample[i] = pll(comp[i])
    locked[i] = pll.locked()

from HDLC import HDLC

class NRZI:

    def __init__(self):

        self.state = False

    def __call__(self, x):
        
        result = (x == self.state)
        self.state = x
        return result


nrzi = NRZI()
hdlc = HDLC()

decoded = None

pll_data = [(b << 2) | (s << 1) | l for b,s,l in zip(comp, sample, locked)]

def cformat(data):
    for i in range(0, len(data), 32):
        for j in range(32):
            if i + j == len(data): break
            print("{:2d}".format(data[i + j]), end=",")
        print()
    print()

cformat(pll_data)

for b,s,l in zip(comp, sample, locked):
    if s:
        packet = hdlc(nrzi(b), l)
        if packet is not None:
            print("uint8_t packet[] = {\n    ", end='')
            count = 0
            for x in packet[1]:
                print('0x%02x, ' % ord(x), end='')
                count += 1
                if count == 16:
                    print('\n    ', end='')
                    count = 0
            print("\n};")
            print(len(packet[1]))
            decoded = packet[1]
            print("0x%04x" % packet[0])

/usr/lib64/python3.7/site-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/usr/lib64/python3.7/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/usr/lib64/python3.7/site-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(s

 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 3, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 7, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7,
 5, 5, 1, 1, 1, 1, 1, 1, 1, 1,

 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 7, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,

 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5,
 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

...

 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5,
 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 3, 1, 1, 1, 1, 1, 1,

 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 7, 5, 5, 5, 5, 5,

 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5,
 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5,
 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 5, 7, 5, 5, 5,

 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5,
 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 5, 5, 5, 5, 5, 5, 5,

 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5,
 5, 5, 7, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 3, 1,

The data above represents the PLL output from 6 seconds of output, along with the decodable frames from that PLL data.

## Vivado HLS

We are going to start with a new project.  We are only going to implement the HDLC decoding in the FPGA.

 1. Start Vivado HLS.
    ```bash
    vivado_hls
    ```
 1. Create a new project under the hdlc directory call HLS.
 1. Create a top-level function called hdlc_top.
 1. Create 5 new files:
    * [hdlc_top.hpp](HLS/hdlc_top.hpp)
    * [hdlc_top.cpp](HLS/hdlc_top.cpp)
    * [hdlc.hpp](HLS/hdlc.hpp)
    * [crc.hpp](HLS/crc.hpp)
    * [nrzi.hpp](HLS/nrzi.hpp)    
 1. Create a new test bench:
    * [hdlc_test.cpp](HLS/hdlc_test.cpp)
    * [pdll_data.hpp](HLS/dpll_data.cpp)
 

### CRC

The CRC code is implemented using a 16-bit shift register.  It is completely pipelined.  Note the use of the FIFO_SRL core, the pipelining of the computation, and the inlining of all of the components.

### HDLC Top-level Interface

Before we address the HDLC implementation, lets look at the top-level function, which implements the AXI interfaces we will be using.

```c++
typedef ap_axiu<8,0,0,0> odata_type;
typedef ap_axiu<3,0,0,0> idata_type;
typedef hls::stream<odata_type> ostream_type;
typedef hls::stream<idata_type> istream_type;
typedef ap_uint<1> cancel_type;

void hdlc_top(ostream_type& output, istream_type& input, cancel_type cancel);
```

This is using a streaming interface for the input and the output.  However, here the input and output data is not symmetical.  The HDLC data is going to contain a lot of noise which will be removed.  Nothing is decoded when the PLL is not locked.  Even when the PLL is locked, much of the data may be noise.

This is why we much have a cancellation input.  We will be reading the output asynchronously while feeding data in.  When the input is complete, we will send a cancel command to cause the HDLC to allow us to close the DMA stream.

Getting this interface -- and the code to implement the interface -- right in Vivado HLS was a rather frustrating exercise.  While it seems that one can send a value to a stream with the "keep" flag set to 0, in practice doing so causes a deadlock somewhere.  The module only works reliably when only elements with the keep flag set are sent through the stream interface.


### HDLC Implementation

And now for the HDLC implementation.  This is a single function which returns a byte of `output` data along with a `valid` flag when data has been decoded.  The input is a bit of data, the PLL lock indicator, and the cancel flag.  
When a packet is complete or aborted, the last byte contains the result code and the `last` flag is set.  If `cancel` is set, the last byte is `STATUS_USER_CANCEL` and `last` is set.

The `cancel` interface allows us to use asyncio in Python for DMA and to cancel the DMA operation.  This is important for HDLC because the amount of data returned is unknown.  Up to this point, the FPGA input and output was always symmetrical so the amount of data expected was known.

The HDLC code is implemented as a state machine with three states: IDLE, SYNC, RECEIVE.  The CRC calculator is reset every time the FSM enters SYNC, and accumulates while in receive.

This is quite a bit different than the Python code or the C++ code in the TNC3 firmware.

```c++
#include <ap_int.h>
#include "crc.hpp"

typedef ap_uint<8> output_type;
typedef ap_uint<1> valid_type;
typedef ap_uint<1> last_type;
typedef ap_uint<1> input_type;
typedef ap_uint<1> lock_type;
typedef ap_uint<1> cancel_type;

const ap_uint<8> STATUS_OK{0x01};
const ap_uint<8> STATUS_USER_CANCEL{0x02};
const ap_uint<8> STATUS_FRAME_ABORT{0x04};
const ap_uint<8> STATUS_FRAME_ERROR{0x08};
const ap_uint<8> STATUS_NO_CARRIER{0x10};
const ap_uint<8> STATUS_CRC_ERROR{0x20};

const ap_uint<16> VALID_CRC = 0xf0b8;

inline void hdlc(
	output_type& output, valid_type& valid_, last_type& last,
	input_type input, lock_type pll_lock, cancel_type cancel)
{
	enum class State {IDLE, SYNC, RECEIVE};

	static State state{State::IDLE};

	static ap_uint<8> buffer{0};

	static ap_uint<3> bits{0};
	static ap_uint<3> ones{0};
	static ap_uint<1> flag{0};
	static crc16_ccitt crc;

	ap_uint<1> valid = 0;

	output = 0;
	last = 0;

#pragma HLS PIPELINE II=2

	if (cancel) {
		state = State::IDLE;
		buffer = 0;
		bits = 0;
		ones = 0;
		flag = 0;
		output = STATUS_USER_CANCEL;
		valid = 1;
		last = 1;
	} else {
		if (pll_lock) {
			if (ones == 5) {
				if (input) {
					// flag byte
					flag = 1;
				} else {
					// bit stuffing...
					flag = 0;
					ones = 0;
					valid_ = valid;
					return;
				}
			}

			buffer >>= 1;
			buffer[7] = input;
			bits += 1;						// Free-running until Sync byte.
			if (input) {
				++ones;
			} else {
				ones = 0;
			}

			HDLC_FSM: switch(state) {

			case State::IDLE:
				if (flag && buffer == 0x7E) {
					state = State::SYNC;
					crc.reset();			// Reset at transition to sync.
					ones = 0;
					flag = 0;
					bits = 0;
				}
				break;

			case State::SYNC:
				if (bits == 0) {	// 8th bit.
					if (flag) {
						if (buffer != 0x7E) {
							state = State::IDLE;
						}
						buffer = 0;
					} else {
						// Start of frame data.
						state = State::RECEIVE;
						output = buffer;
						valid = 1;
						last = 0;
						crc_loop_1: for (int j = 0; j != 8; ++j) {
							crc.compute(buffer[0]);
							buffer >>= 1;
						}
					}
					flag = 0;
					bits = 0;
				}
				break;

			case State::RECEIVE:
				if (bits == 0) {	// 8th bit.
					valid = 1;
					if (flag) {
						last = 1;
						switch (buffer) {
						case 0x7E:
							// End of frame
							output = crc.value() == VALID_CRC ? STATUS_OK : STATUS_CRC_ERROR;
							state = State::SYNC;
							crc.reset();			// Reset at transition to sync.
							break;
						case 0xFE:
						case 0x7F:
							// Abort
							state = State::IDLE;
							output = STATUS_FRAME_ABORT;
							break;
						default:
							// Framing error
							state = State::IDLE;
							output = STATUS_FRAME_ERROR;
							break;
						}
					} else {
						state = State::RECEIVE;
						output = buffer;
						last = 0;
						crc_loop_2: for (int j = 0; j != 8; ++j) {
							crc.compute(buffer[0]);
							buffer >>= 1;
						}
					}
					valid = 1;
					flag = 0;
					bits = 0;
				}
			}
		} else {
			// PLL unlocked.
			if (state == State::RECEIVE) {
				output = crc.value() == VALID_CRC ? STATUS_OK : STATUS_NO_CARRIER;
				valid = 1;
				last = 1;
			}
			state = State::IDLE;
		}

		if (valid) {
			buffer = 0;
		}
	}

	valid_ = valid;
}
```


### C++11

We need to add compilation flags for use during simulation and synthesis.  Right click on the "HLS" project name in the Explorer window on the right side of the Vivado HLS UI and select "Project Settings...".  In the *Project Settings* window, select the *Similation* tab.  Then select the "hdlc_test.cpp" file.  Click the *Edit CFLAGS* button and add "-std=c++11" to the flags.  Go the to *Synthesis* tab, highlight the "hdlc_top.cpp" file and make the same change.

-----

Once the code and test bench are written, we need to run the C simulation, C synthesis, C/RTL co-simulation, then package the IP.  The two simulation steps run our test bench.  This verifies that the code will sythesize properly and that it functions properly.

Once the IP is packaged, we are done in HLS.

## Vivado

We will now switch over to Vivado and create a block design.  These steps should start to feel very familiar to you by now.

 1. Start Vivado and create a new project.
 1. Give it a path -- in our case `afsk-demodulator-pynq/hdlc` and the name `Vivado`.
 1. Select the `RTL Project` project type.
 1. In the "Default Part" screen, switch to the "Boards" tab. Select the your board from the list.
 1. Click "Finish".
 
With the new project open in Vivado, we need to create a block design.  We are going to follow the exact some procedure we did in the first three.

 1. On the right side, in the Flow Navigator, select *Create Block Diagram*.
 1. Use the default name, design_1.
 1. Go into Tools|Settings.
    1. In the settings dialog, choose IP|Repository.
    1. Select "+" to add a repository.
    1. Add hdlc/HLS as a repository.  You should see that it has 1 IP called `hdlc_top` in there.
    1. When done, click "OK".
 1. In the Diagram view (main window) select "+" to add IP.
 1. Add the Zynq processing system and run block automation.
 1. When done, enable the *S AXI HP0 interface* and *IRQ_F2P* in the Zynq block.
    1. Right-click on the Zynq block and select *Customize Block*.
    1. In the Page Navigator pane, select *PS/PL Configuration*
    1. Expand the *HP Slave AXI Interface* section.
    1. Enable *S AXI HP0 Interface*.
    1. In the Page Navigator Select *Interrupts*.
    1. Enable *Fabric Interrupts* and expand the selection.
    1. Expand *PL-PS Interrupt Ports*.
    1. Enable *IRQ_F2P*
    1. Click "OK"
 1. Add an AXI Direct Memory Access, AXI Interrupt Controller, and the hdlc IP.
 1. Open the AXI Direct Memory Access, disable both scatter/gather and the read channel.
 1. Open the AXI Interrupt Controller, change the "Interrupt Output Connection" from *Bus* to *Single*.
 1. Wire up the HDLC module to the AXI Direct Memory Access and run connection automation.
    * A few additional modules are added: AXI SmartConnect, AXI Interconnect, and Processor System Reset
 1. Connect the *s2mm_introut* port on the AXI Direct Memory Access to the *intr* port on the AXI Interrupt Controller.
 1. Connect the *irq* port on the AXI Interrupt Controller to the *IRQ_F2P* port on the Zynq.
![BlockDesign](BlockDesign.png)
 1. Rename the hdlc block to "hdlc" and the DMA block to "dma".
 1. Combine the hdlc and dma blocks into a hierarchy called "demodulator" (select both blocks, then right click and select "Create Hierarchy...").
 1. Generate the HDL wrapper by clicking on the design in the Sources box, right clicking, and selecting "Generate HDL Wrapper".
 1. Generate the bitstream. Again, this will take some time.
 1. Export the block design (File|Export|Export Block Design...)
 1. Collect the following files:
    - Vivado.srcs/sources_1/bd/design_1/hw_handoff/design_1.hwh
    - Vivado.runs/impl_1/design_1_wrapper.bit
    - design_1.tcl
    * rename these file to "hdlc{ext}" so that you have hdlc.bit, hdlc.tcl and hdlc.hwh
 1. On the mounted Pynq filesystem, copy these files to `afsk-demodulator-pynq/hdlc/Pynq/`.
 
    ```bash
    cp hdlc.* /var/run/media/${USER}/PYNQ/jupyter_notebooks/afsk-demodulator-pynq/hdlc/Pynq/
    ```
    
 1. You can now jump to the Jupyter notebook on the Pynq device.